In [1]:
from keras import regularizers
from keras.layers import Input, Dense
from keras.models import Model

encoding_dim = 32

input_img = Input(shape=(784,))
# add a Dense layer with a L1 activity regularizer
encoded = Dense(encoding_dim, activation='relu',
                activity_regularizer=regularizers.l1(10e-5))(input_img)
decoded = Dense(784, activation='sigmoid')(encoded)

autoencoder = Model(input_img, decoded)

Using TensorFlow backend.


create a separate encoder model:

In [2]:
# this model maps an input to its encoded representation
encoder = Model(input_img, encoded)


the decoder model:

In [3]:
# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))

we'll configure our model to use a per-pixel binary crossentropy loss, and the Adadelta optimizer:

In [4]:
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

We're using MNIST digits, and we're discarding the labels

In [5]:
from keras.datasets import mnist
import numpy as np
(x_train, _), (x_test, _) = mnist.load_data()

We will normalize all values between 0 and 1 and we will flatten the 28x28 images into vectors of size 784.

In [6]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print (x_train.shape)
print (x_test.shape)

(60000, 784)
(10000, 784)


train our autoencoder for 5 epochs:

In [7]:
autoencoder.fit(x_train, x_train,
                epochs=5,
                batch_size=32,
                shuffle=True,
                validation_data=(x_test, x_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 10s 164us/step - loss: 0.5564 - val_loss: 0.4613
Epoch 2/5
60000/60000 [==============================] - 9s 151us/step - loss: 0.4131 - val_loss: 0.3774
Epoch 3/5
60000/60000 [==============================] - 9s 152us/step - loss: 0.3560 - val_loss: 0.3392
Epoch 4/5
60000/60000 [==============================] - 9s 152us/step - loss: 0.3278 - val_loss: 0.3186
Epoch 5/5
60000/60000 [==============================] - 9s 152us/step - loss: 0.3117 - val_loss: 0.3061
